## 初期設定

In [11]:
# --- Notebook初期設定 ---
%load_ext autoreload
%autoreload 2
import src.config as cfg
import src.data_loader as dl
import src.cleaning_utils as cu
print("🔁 autoreload 有効化完了")
import yaml
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from collections import defaultdict
import hashlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
🔁 autoreload 有効化完了


In [12]:
# 設定読み込み
settings = cfg.load_settings("setting.yaml")
display(settings)

{'project_name': 'stock_screening',
 'data_path': {'raw': './data/raw',
  'interim': './data/interim',
  'processed': './data/processed'},
 'files': {'raw': ['fy-balance-sheet.csv',
   'fy-cash-flow-statement.csv',
   'fy-profit-and-loss.csv',
   'fy-stock-dividend.csv'],
  'reference': ['CodeData.csv'],
  'interim': None,
  'processed': None},
 'years': [2010,
  2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2020,
  2021,
  2022,
  2023,
  2024,
  2025],
 'na_values': [''],
 'output': {'base_path': './output'}}

## 本ファイルの説明
データが最低限の品質基準を満たしているかを網羅的に確認します。


### データのロードの確認

<details>
<summary><b>結果</b></summary>

ファイルの重複や欠損はなく、正しくロードされました。
</details>

In [ ]:
print("")
print("ファイル毎にデータをロードします")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"],[""]
)
#display(df_DATAs_BY_ALL_FILEs)

print("")
print("エラーなくロードできているようです。次に欠けてるファイルがないか確認します")
for (filename,year), df in df_DATAs_BY_ALL_FILEs.items():
    if df.empty:
        print(file,year,"データフレームが空です")
    else:
        #print("OK")
        continue

print("")
print("欠けているファイルはないようです、次に重複ファイルがないか確認します")
def df_hash(df, ignore_order=False):
    data_only = df.copy().fillna("").astype(str)
    if ignore_order:
        # 行順も無視（全列ソートしてから比較）
        data_only = data_only.sort_values(by=data_only.columns.tolist()).reset_index(drop=True)
    # 列名を無視して内容だけでハッシュを生成
    h = hashlib.md5(pd.util.hash_pandas_object(data_only, index=True).values)
    return h.hexdigest()
hash_map = defaultdict(list)
for (fname, year), df in df_DATAs_BY_ALL_FILEs.items():
    h = df_hash(df, ignore_order=False)  # ←行順を無視したいなら True に
    hash_map[h].append((fname, year))
duplicates = {h: keys for h, keys in hash_map.items() if len(keys) > 1}
if duplicates:
    print("🔍 同じ内容のデータフレーム（ヘッダー無視）が見つかりました：\n")
    for h, key_group in duplicates.items():
        print(f"ハッシュ: {h}")
        for fname, year in key_group:
            print(f"  - {fname}（{year}）")
        print()
else:
    print("✅ 重複データフレームはありません。")

print("")
print("重複しているファイルはないようです。")
print("")
print("データロードの確認を終了します。")



ファイル毎にデータをロードします

エラーなくロードできているようです。次に欠けてるファイルがないか確認します

欠けているファイルはないようです、次に重複ファイルがないか確認します
✅ 重複データフレームはありません。

重複しているファイルはないようです。
データロードの確認を終了します。


### 欠損値表現の確認と処理

<details>
<summary><b>結果</b></summary>

</details>

In [ ]:
# 欠損値の表現確認
print("")
print("ファイル毎にデータを取得します")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"],[""]
)
#display(df_DATAs_BY_ALL_FILEs)

print("")
print("ファイルごとに各文字列の個数をカウントします")
#"nan", "na", "n/a", "null", "-", "--", "none", "0"," ",""

df_placeholder_counts = pd.DataFrame(
    columns=["col","nan","na","n/a","-","--","none","0","space","","alphabet"],
)
display(df_placeholder_counts)
for (filename,year), df in df_DATAs_BY_ALL_FILEs.items():
    counts = []
    for col in df:
        ser = df[col].fillna("")
        ser_str = ser.astype(str)
        #display(ser_str)
        counts.append(
            (col,
            (ser_str == 'nan').sum(),
            (ser_str == 'na').sum(),
            (ser_str == 'n/a').sum(),
            (ser_str == '-').sum(),
            (ser_str == '--').sum(),
            (ser_str == 'none').sum(),
            (ser_str == '0').sum(),
            (ser_str == ' ').sum(),
            (ser_str == '').sum(),
            ser_str.str.contains('[A-Za-z]', na=False).sum())
        )
        df_placeholder_counts.loc[df_placeholder_counts.index.max()+1] = counts
        display(df_placeholder_counts)



### データ型の整合性確認

<details>
<summary><b>結果</b></summary>

</details>

### 初期品質の概要確認

<details>
<summary><b>結果</b></summary>

</details>


In [ ]:
# 財務データ 結合前
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_rows', 60)

def get_file_info(df_datas_by_all_files):
    df_file_info = pd.DataFrame(columns=["file","year","code_counts","year_counts","column_counts"])
    for file, year in df_datas_by_all_files.keys():
        df = df_datas_by_all_files[(file,year)]
        # len(df_file_info)を次の新しい行のインデックスとして明示的に指定します。
        df_file_info.loc[len(df_file_info)] = [
            file, year, df["コード"].nunique(), df["年度"].nunique(),df.columns.nunique()
        ]
    return df_file_info

print("")
print("ファイル毎にデータを取得します")
df_DATAs_BY_ALL_FILEs = dl.load_data_by_files(
    settings["data_path"]["raw"],settings["years"],settings["files"]["raw"],["-","","0"]
)
#display(df_DATAs_BY_ALL_FILEs)

print("")
print("企業コード数、エンドの数、列数をファイルごとにまとめます。")
df_file_info = get_file_info(df_DATAs_BY_ALL_FILEs)
#display(df_file_info)

print("")
print("各ファイルの年推移を可視化します。")
"""fig = px.line(df_file_info,x="year",y="code_counts",color="file")
fig.show()
fig = px.line(df_file_info,x="year",y="year_counts",color="file")
fig.show()
fig = px.line(df_file_info,x="year",y="column_counts",color="file")
fig.show()"""

print("")
print("列数はOKです")
print("最新年のファイルに登録されている年度を表示します。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df["年度"].unique())

print("")
print("最新年のファイルに過去のデータがあります。")
print("最新年に登録されている過去の年度と過去のデータが同じか確認します")
print("まずは、年度を限定して、コードの一覧を見ます")
#display(df[df["年度"]=="2024/12"])

print("")
print("コードを限定し同じかどうか調べます。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df[df["コード"]=="130A"])
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2024)]
#display(df[df["コード"]=="130A"])

print("")
print("違っています。これは更新データと思われます。")
print("最新年度にある過去のデータを取得し、過去のデータを更新します")
df_DATAs_BY_ALL_FILEs = dl.update_duplicated(df_DATAs_BY_ALL_FILEs, 2025)
#display(df_DATAs_BY_ALL_FILEs)

print("")
print("正しく処理が行われ、最新年に登録されている過去の年度と過去のデータが同じか確認します。")
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2025)]
#display(df[df["コード"]=="130A"])
df = df_DATAs_BY_ALL_FILEs[(settings["files"]["raw"][0],2024)]
#display(df[df["コード"]=="130A"])

print("")
print("最後に最新年度にある過去のデータを消去します。")
cutoff_date = pd.to_datetime("2025-01-01")
for file, year in df_DATAs_BY_ALL_FILEs.keys():
    if year == 2025:
        df = df_DATAs_BY_ALL_FILEs[(file,year)]
        df = df[df["年度"] >= cutoff_date]
        df_DATAs_BY_ALL_FILEs[(file,year)] = df
print("")
print("各ファイルの年推移をもう一度可視化し、改善していることを確かめます。") 
df_file_info_after = get_file_info(df_DATAs_BY_ALL_FILEs)
df_file_info_after["区分"] = "処理後"
df_file_info_after = df_file_info_after.set_index(["file", "year"]).sort_index()
df_file_info["区分"] = "処理前"
df_file_info = df_file_info.set_index(["file", "year"]).sort_index()
df_compare = pd.concat([df_file_info,df_file_info_after]).sort_index().reset_index()
#display(df_compare)
fig = px.line(df_compare,x="year",y="code_counts",color="区分")
fig.show()


In [ ]:
# 財務データ ファイルごとの結合
df_DATAs_BY_FILEs = dl.load_yearly_data(settings["data_path"], settings["years"], settings["files"],)
target_file = settings["files"][0]
display(df_DATAs_BY_FILEs[target_file][df_DATAs_BY_FILEs[target_file]["コード"]=="130A"])

In [ ]:
# 財務データ　全結合
df_ALL_DATAs = dl.merge_all_data(df_DATAs_BY_FILEs)
print(df_ALL_DATAs.shape)
#display(df_ALL_DATAs)
display(df_ALL_DATAs["コード"].unique())
display(df_ALL_DATAs["年度"].nunique())

In [ ]:
# 上場企業の情報
df = dl.load_code_list_info(settings["data_path"], settings["files_reference"][0])
print(settings["files_reference"][0], df.shape)
display(df)

### データ型の整合性チェック

### 初期品質の概要把握